In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical

# Load and preprocess the dataset
def load_and_preprocess_data():
    # Load data
    X_train = np.loadtxt("../../Datasets/CAPP Dataset/SubjectDependent50PercentOverlap/X_train.txt")
    y_train = np.loadtxt("../../Datasets/CAPP Dataset/SubjectDependent50PercentOverlap/y_train.txt")
    X_test = np.loadtxt("../../Datasets/CAPP Dataset/SubjectDependent50PercentOverlap/X_test.txt")
    y_test = np.loadtxt("../../Datasets/CAPP Dataset/SubjectDependent50PercentOverlap/y_test.txt")

    # Reshape data (assuming 100 time steps and 9 features)
    X_train = X_train.reshape(X_train.shape[0], 100, 9)
    X_test = X_test.reshape(X_test.shape[0], 100, 9)

    # Normalize features
    scaler = StandardScaler()
    X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
    X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])
    X_train_scaled = scaler.fit_transform(X_train_reshaped).reshape(X_train.shape)
    X_test_scaled = scaler.transform(X_test_reshaped).reshape(X_test.shape)

    # Find the actual number of unique classes
    all_labels = np.unique(np.concatenate((y_train, y_test)))
    num_classes = len(all_labels)
    print(f"Unique labels: {all_labels}")
    print(f"Number of classes: {num_classes}")

    # Create a mapping from original labels to consecutive integers
    label_map = {label: i for i, label in enumerate(all_labels)}

    # Apply the mapping to y_train and y_test
    y_train_mapped = np.array([label_map[label] for label in y_train])
    y_test_mapped = np.array([label_map[label] for label in y_test])

    # One-hot encode the mapped labels
    y_train_onehot = to_categorical(y_train_mapped, num_classes=num_classes)
    y_test_onehot = to_categorical(y_test_mapped, num_classes=num_classes)

    print(f"One-hot encoded label shape: {y_train_onehot.shape[1]}")

    return X_train_scaled, y_train_onehot, X_test_scaled, y_test_onehot, num_classes, label_map


# Create the TFLite Micro compatible model
def create_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Train the model
def train_model(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=32):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping],
                        verbose=1)
    return history

def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test accuracy: {accuracy:.4f}")

    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    # Convert back to original labels for the classification report
    inv_label_map = {v: k for k, v in label_map.items()}
    y_pred_original = np.array([inv_label_map[label] for label in y_pred_classes])
    y_test_original = np.array([inv_label_map[label] for label in y_test_classes])

    print(classification_report(y_test_original, y_pred_original, digits=5))

# Convert to TFLite
def convert_to_tflite(model, X_test):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8

    def representative_dataset_gen():
        for input_value in tf.data.Dataset.from_tensor_slices(X_test).batch(1).take(100):
            yield [input_value]

    converter.representative_dataset = representative_dataset_gen
    tflite_model = converter.convert()

    with open('subject_dependent_onehot_tflite_model.tflite', 'wb') as f:
        f.write(tflite_model)
    print("TFLite model saved as 'subject_dependent_onehot_tflite_model.tflite'")

# Load and preprocess data
X_train, y_train, X_test, y_test, num_classes, label_map = load_and_preprocess_data()
print("num_classes: ", num_classes)

# Split training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Create and train the model
input_shape = X_train.shape[1:]
CNN_MLP = create_model(input_shape, num_classes)
history = train_model(CNN_MLP, X_train, y_train, X_val, y_val)

# Evaluate the model
evaluate_model(CNN_MLP, X_test, y_test)


2024-09-27 11:59:25.289699: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 11:59:25.300497: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 11:59:25.303746: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-27 11:59:25.312584: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-27 11:59:25.813125: W tensorflow/compiler/tf2

Unique labels: [ 0.  1.  2.  3.  6.  7.  8.  9. 10. 11. 12. 13. 14. 19. 20. 21. 22. 23.
 25. 26. 27.]
Number of classes: 21
One-hot encoded label shape: 21
num_classes:  21


I0000 00:00:1727416788.873022  212741 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727416788.907511  212741 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727416788.910396  212741 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727416788.913902  212741 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Epoch 1/50


I0000 00:00:1727416790.170603  213079 service.cc:146] XLA service 0x72a8b80053d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727416790.170631  213079 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-09-27 11:59:50.200155: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-27 11:59:50.295968: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-09-27 11:59:51.465450: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_12', 52 bytes spill stores, 52 bytes spill loads

2024-09-27 11:59:51.554429: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function

 151/2390 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.1527 - loss: 2.7587

I0000 00:00:1727416792.405861  213079 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2355/2390 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step - accuracy: 0.5234 - loss: 1.5238

2024-09-27 11:59:55.347113: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_12', 4 bytes spill stores, 4 bytes spill loads

2024-09-27 11:59:55.555538: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_13', 4 bytes spill stores, 4 bytes spill loads



2390/2390 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.5253 - loss: 1.5174 - val_accuracy: 0.7494 - val_loss: 0.7633
Epoch 2/50
2390/2390 ━━━━━━━━━━━━━━━━━━━━ 2s 681us/step - accuracy: 0.7616 - loss: 0.7293 - val_accuracy: 0.7901 - val_loss: 0.6479
Epoch 3/50
2390/2390 ━━━━━━━━━━━━━━━━━━━━ 2s 800us/step - accuracy: 0.7962 - loss: 0.6170 - val_accuracy: 0.8093 - val_loss: 0.5699
Epoch 4/50
2390/2390 ━━━━━━━━━━━━━━━━━━━━ 2s 834us/step - accuracy: 0.8215 - loss: 0.5374 - val_accuracy: 0.8321 - val_loss: 0.5064
Epoch 5/50
2390/2390 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8389 - loss: 0.4825 - val_accuracy: 0.8509 - val_loss: 0.4626
Epoch 6/50
2390/2390 ━━━━━━━━━━━━━━━━━━━━ 2s 963us/step - accuracy: 0.8488 - loss: 0.4485 - val_accuracy: 0.8568 - val_loss: 0.4299
Epoch 7/50
2390/2390 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8626 - loss: 0.4152 - val_accuracy: 0.8698 - val_loss: 0.3952
Epoch 8/50
2390/2390 ━━━━━━━━━━━━━━━━━━━━ 2s 682us/step - accuracy: 0.8704 - loss: 0.3938 - v

In [3]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(CNN_MLP)
tflite_model = converter.convert()

# Save the TFLite model
with open("GeneratedTFLiteFilesAndOGModels/conv_mlp_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TensorFlow Lite model has been saved as 'conv_mlp_model.tflite'")

# Optional: Save the Keras model
CNN_MLP.save("GeneratedTFLiteFilesAndOGModels/conv_mlp_model.h5")
print("Keras model has been saved as 'conv_mlp_model.h5'")

try:
    from everywhereml.code_generators.tensorflow import convert_model

    c_header = convert_model(CNN_MLP, X_test, y_test, model_name='conv_mlp_model')

    with open("GeneratedHeaderFiles/conv_mlp_model.h", "w") as file:
        file.write(c_header)

    print("C header file has been saved as GeneratedHeaderFiles/conv_mlp_model.h'")
except ImportError:
    print("everywhereml library not found. Skipping C header file generation.")

INFO:tensorflow:Assets written to: /tmp/tmp3zvpot_6/assets


INFO:tensorflow:Assets written to: /tmp/tmp3zvpot_6/assets


Saved artifact at '/tmp/tmp3zvpot_6'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 21), dtype=tf.float32, name=None)
Captures:
  126077390594896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126077390598592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126077390601056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126077386150080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126077386149376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126077386151664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126077386149728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126077386153600: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1727416952.079192  212741 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1727416952.079202  212741 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-09-27 12:02:32.079310: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp3zvpot_6
2024-09-27 12:02:32.079603: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-09-27 12:02:32.079611: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp3zvpot_6
2024-09-27 12:02:32.082100: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-09-27 12:02:32.097646: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp3zvpot_6
2024-09-27 12:02:32.102423: I tensorflow/cc/saved_model/loader.cc:462] SavedModel load for tags { serve }; Status: success: OK. Took 23115 microseconds.


TensorFlow Lite model has been saved as 'conv_mlp_model.tflite'
Keras model has been saved as 'conv_mlp_model.h5'
INFO:tensorflow:Assets written to: /tmp/tmpucrgzuik/assets


INFO:tensorflow:Assets written to: /tmp/tmpucrgzuik/assets


Saved artifact at '/tmp/tmpucrgzuik'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 21), dtype=tf.float32, name=None)
Captures:
  126077390594896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126077390598592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126077390601056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126077386150080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126077386149376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126077386151664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126077386149728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126077386153600: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1727416952.314021  212741 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1727416952.314031  212741 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-09-27 12:02:32.314146: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpucrgzuik
2024-09-27 12:02:32.314445: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-09-27 12:02:32.314454: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpucrgzuik
2024-09-27 12:02:32.316996: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-09-27 12:02:32.332572: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpucrgzuik
2024-09-27 12:02:32.337355: I tensorflow/cc/saved_model/loader.cc:462] SavedModel load for tags { serve }; Status: success: OK. Took 23211 microseconds.


C header file has been saved as GeneratedHeaderFiles/conv_mlp_model.h'
